In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('titanic3.csv')
data.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI"
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"


In [3]:
# Preprocessing data

features = data[['pclass', 'body', 'boat']]
features['boat'] = features['boat'].fillna(0)
features[features['boat'] != 0] = 1

processed_data = pd.get_dummies(features)

processed_data['body'] = processed_data['body'].fillna(0)
processed_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

,pclass,body,boat
0,1,1.0,1
1,1,1.0,1
2,1,0.0,0
3,1,135.0,0
4,1,0.0,0


In [4]:
np.any(processed_data.isnull())

False

In [5]:
for col in processed_data.columns:
    processed_data[processed_data[col].isnull()] = processed_data[col].mean()

np.any(processed_data.isnull())

False

In [648]:
y = np.array(data.pop('survived'))
X = np.array(processed_data)

In [649]:
from sklearn.model_selection import train_test_split

# split data into trian, validation and test
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

data_valid, y_train = y_train[:len(y_train) // 2], y_train[len(y_train) // 2:]
target_valid, y_test = y_test[:len(y_test) // 2], y_test[len(y_test) // 2:]

In [650]:
# here are the models with best hyperparameters for every model
best_models = []

In [651]:
# model 1

from sklearn.neighbors import KNeighborsClassifier

neighbors = [1, 3, 5, 7, 8]

folds = 5
fold_size = len(X_train) // folds
models_and_scores = []

# cross validation
for fold in range(folds):
    mask = np.ones(len(X_train), dtype=bool)
    mask[fold * fold_size: (fold + 1) * fold_size] = False
    
    X_train_ = X_train[mask]
    X_test_ = X_test[mask]
    y_train_ = X_train[~mask]
    y_test_ = X_test[~mask]
    
    knns = [
        KNeighborsClassifier(n_neighbors=neighbor).fit(X_train_, X_test_)
        for neighbor in neighbors
    ]
    models_and_scores.extend([(knn, knn.score(y_train_, y_test_)) for knn in knns])

best_model, best_score = None, 0
for model, score in models_and_scores:
    if score > best_score:
        best_model, best_score = model, score

print(f'Accuracy score on train data is {best_score}')
print(f'Accuracy score on validation data is {best_model.score(data_valid, target_valid)}')

best_models.append(best_model)

Accuracy score on train data is 1.0
Accuracy score on validation data is 0.9770992366412213


In [652]:
# model 2

# ignore warings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold

grid_params = {'C': [000.1, 00.1, 0.1, 10]}

grid = GridSearchCV(LogisticRegression(), param_grid=grid_params, cv=KFold(shuffle=True))
grid.fit(X_train, X_test)

best_score = grid.best_score_
best_model = grid.best_estimator_

print(f'Accuracy score on train data is {best_score}')
print(f'Accuracy score on validation data is {best_model.score(data_valid, target_valid)}')

best_models.append(best_model)

Accuracy score on train data is 0.9694267515923567
Accuracy score on validation data is 0.9770992366412213


In [653]:
warnings.simplefilter(action='default')

In [654]:
# model 3

# could not find hyperparameters or this model does not have them

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

scores = cross_val_score(GaussianNB(), X_train, X_test, cv=5)

best_score = max(scores)
best_model = GaussianNB()
best_model.fit(X_train, X_test)

print(f'Accuracy score on train data is {best_score}')
print(f'Accuracy score on validation data is {best_model.score(data_valid, target_valid)}')

best_models.append(best_model)

Accuracy score on train data is 0.9936305732484076
Accuracy score on validation data is 0.9770992366412213


In [655]:
# test all different best models on test data

[(model.__class__.__name__, model.score(y_train, y_test)) for model in best_models]

[('KNeighborsClassifier', 0.9923664122137404),
 ('LogisticRegression', 0.9923664122137404),
 ('GaussianNB', 0.9923664122137404)]